<img src = "https://raw.githubusercontent.com/PauliBCN/Capstone/master/Toronto.JPG" width = 350>
<h1 align=center><font size = 5>Toronto</font></h1>

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df[0].head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df2 = df[0]
df2 = df2[df2.Borough != 'Not assigned']
for i, row in df2.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
df2.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [4]:
#df3=df2['Postcode','Borough','Neighborhood']
df3=df2.groupby("Postcode")["Neighborhood"].agg(",".join).reset_index(name='new')
df3 = df3.rename(columns={'new':'Neighborhood'})
df3.head(10)

,Postcode,Neighborhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,"Clairlea,Golden Mile,Oakridge"
8,M1M,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,"Birch Cliff,Cliffside West"
